In [1]:
!pip install patool --q
!pip install transformers --q
!pip install emot --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.3 MB/s eta 0:00:00


In [2]:
import emot
import pandas as pd
import patoolib
import re
import nltk
import numpy as np
import matplotlib.pyplot as plt
import torch

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from transformers import AutoTokenizer, AutoModelForSequenceClassification

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import re
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import patoolib

# Define functions
def unrar_file(file_path, output_dir):
    patoolib.extract_archive(file_path, outdir=output_dir)

def hashtags(text):
    hash = re.findall(r"#(\w+)", text)
    return hash

def emoji(text):
    for emot in UNICODE_EMOJI:
        if text == None:
            text = text
        else:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
    return text

def remove_users(tweet):
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    return tweet

def remove_links(tweet):
    tweet = re.sub(r'http\S+', '', tweet)  # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')  # remove [links]
    return tweet

def clean_html(text):
    html = re.compile('<.*?>')  # regex
    return html.sub(r'', text)

def non_ascii(s):
    return "".join(i for i in s if ord(i) < 128)

def lower(text):
    return text.lower()

def email_address(text):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'', text)

def punct(text):
    token = RegexpTokenizer(r'\w+')  # regex
    text = token.tokenize(text)
    text = " ".join(text)
    return text

def removeStopWords(str):
    # select english stopwords
    cachedStopWords = set(stopwords.words("english"))
    # add custom words
    cachedStopWords.update(('and', 'I', 'A', 'http', 'And', 'So', 'arnt', 'This', 'When', 'It', 'many', 'Many', 'so', 'cant',
                            'Yes', 'yes', 'No', 'no', 'These', 'these', 'mailto', 'regards', 'ayanna', 'like', 'email'))
    # remove stop words
    new_str = ' '.join([word for word in str.split() if word not in cachedStopWords])
    return new_str

def remove_(tweet):
    tweet = re.sub('([_]+)', "", tweet)
    return tweet

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))

# Unrar file
rar_file_path = '/content/datasets.rar'
output_directory = '/content'
unrar_file(rar_file_path, output_directory)

# Read CSV file and extract relevant columns
df = pd.read_csv('/content/$OCEAN tweets dataset.csv')[['date', 'tweet']]

# Filter tweets containing '$OCEAN'
df['contains_$OCEAN'] = df['tweet'].str.contains(r'\$OCEAN', case=False, regex=True)
df = df[df['contains_$OCEAN']]

# Group tweets by date
df = df.groupby('date')['tweet'].apply(' '.join).reset_index()
dfSentiment = df.copy()

# Apply data cleaning functions to the tweets
dfSentiment['new_tweet'] = dfSentiment.tweet.apply(emoji)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(remove_users)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(clean_html)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(remove_links)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(non_ascii)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(lower)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(email_address)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(removeStopWords)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(clean_html)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(punct)
dfSentiment['new_tweet'] = dfSentiment.new_tweet.apply(remove_)

# Perform sentiment analysis on the cleaned tweets
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest', from_tf=True)
dfSentiment['sentiment'] = dfSentiment['new_tweet'].apply(lambda x: sentiment_score(x[:512]))


patool: Extracting /content/datasets.rar ...
patool: running /usr/bin/unrar x -- /content/datasets.rar
patool:     with cwd='/content'
patool: ... /content/datasets.rar extracted to `/content'.


All TF 2.0 model weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.


In [5]:
dfSentiment.sentiment.value_counts()

2    576
1    407
0     42
Name: sentiment, dtype: int64

In [6]:
import pandas as pd

# Read the datasets
df_base = pd.read_csv('/content/$OCEAN tweets dataset.csv')
price = pd.read_csv('/content/OCEAN token price dataset.csv')
#df_sentiment = pd.read_csv('/content/test (5).csv') #this is a dump i made for dfSentiment
df_sentiment = dfSentiment.copy()
# Rename the 'Date' column in the 'price' DataFrame
price.rename(columns={'Date': 'date'}, inplace=True)

# Merge price and df_base on 'date' column
result_base = pd.merge(price, df_base, on='date', how='inner')

# Merge price and df_sentiment on 'date' column
result = pd.merge(price, df_sentiment, on='date', how='inner')

# Filter result_base to include tweets containing '$OCEAN'
result_base['contains_$OCEAN'] = result_base['tweet'].str.contains(r'\$OCEAN', case=False, regex=True)
result_base = result_base[result_base['contains_$OCEAN']]

# Calculate the correlation between the price of $OCEAN and the number of tweets containing "$OCEAN".
result_base['Count'] = result_base.groupby('date')['date'].transform('count')
correlation_tweets = result_base.drop_duplicates('date')['Count'].corr(result_base['Close'])
print("Correlation between tweets and stock price:", correlation_tweets)

# Determine the correlation between the price of $OCEAN and the number of likes received by tweets containing "$OCEAN".
correlation_likes = result_base.groupby('date')['likes_count'].transform('sum').corr(result_base['Close'])
print("Correlation between number of likes and stock price:", correlation_likes)

# Establish the correlation between the price of $OCEAN and the number of retweets generated by tweets containing "$OCEAN".
correlation_retweets = result_base.groupby('date')['retweets_count'].transform('sum').corr(result_base['Close'])
print("Correlation between number of retweets and stock price:", correlation_retweets)

# Assess the correlation between the price of $OCEAN and the number of individuals tweeting with the cashtag "$OCEAN".
correlation_users = result_base.groupby('date')['user_id'].nunique().reset_index()['user_id'].corr(result_base['Close'])
print("Correlation between number of individuals tweeting and stock price:", correlation_users)

# Analyze the impact of influential tweets on the price of the OCEAN token.
filtered = result_base[result_base['likes_count'] > result_base['likes_count'].quantile(0.95)]
correlation_influential_tweets = filtered.drop_duplicates('date')['Count'].corr(result_base['Close'])
print("Correlation between influential tweets and stock price:", correlation_influential_tweets)

# Is there a correlation between investor sentiment and the token's price?
correlation = result['sentiment'].corr(result['Close'])
print("Correlation between sentiment and stock price:", correlation)


Correlation between tweets and stock price: 0.44475406929240474
Correlation between number of likes and stock price: 0.6239078203653493
Correlation between number of retweets and stock price: 0.4609170228184592
Correlation between number of individuals tweeting and stock price: -0.3645092710895042
Correlation between influential tweets and stock price: 0.4608633698823815
Correlation between sentiment and stock price: 0.014888413981419773


<ipython-input-6-714464918f83>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_base['Count'] = result_base.groupby('date')['date'].transform('count')
